#Importing Libraries

In [4]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('/kaggle/input/bengali-sms-dataset/bangla_smish.csv')
df

,label,text
0,smish,বাংলাদেশ ব্যাংক থেকে গুরুত্বপূর্ণ বার্তা। জরুর...
1,normal,আপনার দিন কেমন কাটছে?
2,promo,"৪৫৳ ক্যাশব্যাক! ৪৬জিবি @ ৪৫৩৳, ৩০দিন, নিয়ে নাও..."
3,normal,"হ্যালো, কি খবর?"
4,smish,জনতা ব্যাংক অ্যাকাউন্টে সমস্যা হয়েছে। কল করুন...
...,...,...
2282,promo,বোনাস সহ ৩জিবি-৬০টাকা-৩দিন। ডায়াল *১২১*৫৬৯৯# ব...
2283,promo,"নতুন স্যামসাং ফোনে ১০% ডিসকাউন্ট! স্টক সীমিত, ..."
2284,normal,শুভ অসুস্থ্য দিনের শুভেচ্ছা!
2285,normal,আপনার কেমন দিন কাটছে?


In [8]:
df.sample(10)

,label,text
828,normal,তুমি কতটুকু পড়েছো এই সপ্তাহে? অনেক পড়েছি। তুমি?
1036,normal,একটু ভালোবাসা চাই।
1876,promo,বোনাস সহ ৮জিবি-১২০টাকা-৩০দিন। নিতে ডায়াল *১২১*...
2030,promo,বিশেষ অফার: সব বাচ্চাদের পোশাকে ২০% ডিসকাউন্ট।
403,promo,নতুন সিজনে সব পোশাকে ২৫% ডিসকাউন্ট। আজই শপিং ক...
848,promo,সর্বকালের সর্বশ্রেষ্ঠ বাঙালি জাতির পিতা বঙ্গবন...
1910,smish,"অভিনন্দন! আপনি 50,000 টাকা নগদ পুরস্কার জিতেছে..."
713,promo,সিস্টেমের উন্নয়নের জন্য ৮মে রাত ১১:৫৯ থেকে ৯মে...
104,normal,তুমি কি পাহাড়ে যাচ্ছো?
1990,smish,"""মানসিক স্বাস্থ্য সম্পর্কে জানুন! নিজের জন্য স..."


In [9]:
df.shape

(2287, 2)

In [1]:
df.info

NameError: name 'df' is not defined